## Datawarehouse

In [13]:

import pandas as pd
from sqlalchemy import create_engine, Column, Integer, Float, String, Date, ForeignKey, ForeignKeyConstraint
from sqlalchemy.orm import Session, declarative_base, relationship

df = pd.read_csv("/home/FIL_ROUGE/Historical/data/processed/weatherAirSoil.csv")
# df['weather_description'] = df['weather_description'].astype('string')

    # Créer la chaîne de connexion SQLAlchemy
# server = '172.16.11.132'
#     # server = '172.25.0.1'
# database = 'soil_datamart'
# username = 'fidele'
# password = 'admin'
# conn_str = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server'

#     # Créer le moteur SQLAlchemy
# engine = create_engine(conn_str, echo=False)

#     # Créer la session SQLAlchemy
# session = Session(engine)

# conn = engine.connect()
# print("Connexion réussie.")

# df.dtypes


        # Colonnes et tables de dimension
dimension_columns = ['time', 'cloud_cover (%)', 'et0_fao_evapotranspiration (mm)',
                            'temperature_2m (°C)', 'vapour_pressure_deficit (kPa)', 'rain (mm)',
                            'snowfall (cm)', 'wind_speed_10m (m/s)', 'relative_humidity_2m (%)',
                            'pressure_msl (hPa)', 'weather_description']

dimension_tables = ['DimDate', 'DimCloudCover', 'DimEvapotranspiration',
                            'DimTemperature', 'DimVPD', 'DimRain', 'DimSnowfall',
                            'DimWind', 'DimHumidity', 'DimPressure', 'DimWeather']



# Créer les tables de dimension
dimension_dataframes = {}
for column, table in zip(dimension_columns, dimension_tables):
    df_dimension = df[[column]].drop_duplicates().reset_index(drop=True)
    # print(df_dimension)
#     Correction du nom de la colonne générée
    column_name = f"{table.lower()}ID"
    df_dimension.insert(0, column_name, range(1, len(df_dimension) + 1))
    if column == 'weather_description' :
        df['weather_description'] = df['weather_description'].astype(str)
        df['weather_description'] = df['weather_description'].str.encode('utf-8', errors='ignore').str.decode('utf-8', errors='ignore')

        # df['weather_description'] = df['weather_description'].astype(pd.StringDtype())
            
    # df_dimension.to_sql(table, conn, if_exists='append', index=False, method='multi', chunksize=500)
    print(df_dimension.dtypes)
            
            # Ajouter le DataFrame à dimension_dataframes
    dimension_dataframes[table] = df_dimension
# print(dimension_dataframes)


            # if column == 'weather_description' :
            #     print("hop !")
            #     df['weather_description'] = df['weather_description'].astype('string')
            #     print("convertis !")
            #     for index, row in df_dimension.iterrows():
            #         print(row)
            #         weather_obj = DimWeather(weather_description=row['weather_description'])
            #         session.add(weather_obj)



dimdateID     int64
time         object
dtype: object
dimcloudcoverID    int64
cloud_cover (%)    int64
dtype: object
dimevapotranspirationID              int64
et0_fao_evapotranspiration (mm)    float64
dtype: object
dimtemperatureID         int64
temperature_2m (°C)    float64
dtype: object
dimvpdID                           int64
vapour_pressure_deficit (kPa)    float64
dtype: object
dimrainID      int64
rain (mm)    float64
dtype: object
dimsnowfallID      int64
snowfall (cm)    float64
dtype: object
dimwindID                 int64
wind_speed_10m (m/s)    float64
dtype: object
dimhumidityID               int64
relative_humidity_2m (%)    int64
dtype: object
dimpressureID           int64
pressure_msl (hPa)    float64
dtype: object
dimweatherID            int64
weather_description    object
dtype: object


In [8]:
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, Float, Date, ForeignKey, ForeignKeyConstraint
from sqlalchemy.orm import Session, declarative_base, relationship
import sys

Base = declarative_base()

class DimDate(Base):
    __tablename__ = 'DimDate'
    dimdateID = Column(Integer, primary_key=True, autoincrement=True)
    time = Column(Date, unique=True)

class DimHumidity(Base):
    __tablename__ = 'DimHumidity'
    dimhumidityID = Column(Integer, primary_key=True, autoincrement=True)
    relative_humidity_2m = Column(Float)

class DimCloudCover(Base):
    __tablename__ = 'DimCloudCover'
    dimcloudcoverID = Column(Integer, primary_key=True, autoincrement=True)
    cloud_cover = Column(Float)

class DimEvapotranspiration(Base):
    __tablename__ = 'DimEvapotranspiration'
    dimevapotranspirationID = Column(Integer, primary_key=True, autoincrement=True)
    et0_fao_evapotranspiration = Column(Float)

class DimTemperature(Base):
    __tablename__ = 'DimTemperature'
    dimtemperatureID = Column(Integer, primary_key=True, autoincrement=True)
    temperature_2m = Column(Float)

class DimVPD(Base):
    __tablename__ = 'DimVPD'
    dimvpdID = Column(Integer, primary_key=True, autoincrement=True)
    vapour_pressure_deficit = Column(Float)

class DimRain(Base):
    __tablename__ = 'DimRain'
    dimrainID = Column(Integer, primary_key=True, autoincrement=True)
    rain = Column(Float)

class DimSnowfall(Base):
    __tablename__ = 'DimSnowfall'
    dimsnowfallID = Column(Integer, primary_key=True, autoincrement=True)
    snowfall = Column(Float)
    snow_depth = Column(Float)

class DimWind(Base):
    __tablename__ = 'DimWind'
    dimwindID = Column(Integer, primary_key=True, autoincrement=True)
    wind_speed_10m = Column(Float)


class FactAirSoil(Base):
    __tablename__ = 'Fact_Air_Soil'
    fact_ID = Column(Integer, primary_key=True, autoincrement=True)
    time_id = Column(Integer, ForeignKey('DimDate.dimdateID'))
    cloud_cover_id = Column(Integer, ForeignKey('DimCloudCover.dimcloudcoverID'))
    evapotranspiration_id = Column(Integer, ForeignKey('DimEvapotranspiration.dimevapotranspirationID'))
    temperature_id = Column(Integer, ForeignKey('DimTemperature.dimtemperatureID'))
    vpd_id = Column(Integer, ForeignKey('DimVPD.dimvpdID'))
    rain_id = Column(Integer, ForeignKey('DimRain.dimrainID'))
    snowfall_id = Column(Integer, ForeignKey('DimSnowfall.dimsnowfallID'))
    wind_id = Column(Integer, ForeignKey('DimWind.dimwindID'))
    humidity_id = Column(Integer, ForeignKey('DimHumidity.dimhumidityID'))

    # Ajoutez des mesures
    pm10 = Column(Float)
    pm2_5 = Column(Float)
    carbon_monoxide = Column(Float)
    nitrogen_dioxide = Column(Float)
    sulphur_dioxide = Column(Float)
    ozone = Column(Float)
    soil_temperature_0_to_7cm = Column(Float)
    soil_temperature_7_to_28cm = Column(Float)
    soil_temperature_28_to_100cm = Column(Float)
    soil_temperature_100_to_255cm = Column(Float)
    soil_moisture_0_to_7cm = Column(Float)
    soil_moisture_7_to_28cm = Column(Float)
    soil_moisture_28_to_100cm = Column(Float)
    soil_moisture_100_to_255cm = Column(Float)

    # Ajoutez les relations avec les tables de dimension
    time = relationship('DimDate', foreign_keys=[time_id])
    cloud_cover = relationship('DimCloudCover', foreign_keys=[cloud_cover_id])
    evapotranspiration = relationship('DimEvapotranspiration', foreign_keys=[evapotranspiration_id])
    temperature = relationship('DimTemperature', foreign_keys=[temperature_id])
    vpd = relationship('DimVPD', foreign_keys=[vpd_id])
    rain = relationship('DimRain', foreign_keys=[rain_id])
    snowfall = relationship('DimSnowfall', foreign_keys=[snowfall_id])
    wind = relationship('DimWind', foreign_keys=[wind_id])
    humidity = relationship('DimHumidity', foreign_keys=[humidity_id])

    # Définissez les contraintes de clé étrangère ici
    __table_args__ = (
        ForeignKeyConstraint(['cloud_cover_id'], ['DimCloudCover.dimcloudcoverID']),
        ForeignKeyConstraint(['time_id'], ['DimDate.dimdateID']),
        ForeignKeyConstraint(['evapotranspiration_id'], ['DimEvapotranspiration.dimevapotranspirationID']),
        ForeignKeyConstraint(['temperature_id'], ['DimTemperature.dimtemperatureID']),
        ForeignKeyConstraint(['vpd_id'], ['DimVPD.dimvpdID']),
        ForeignKeyConstraint(['rain_id'], ['DimRain.dimrainID']),
        ForeignKeyConstraint(['snowfall_id'], ['DimSnowfall.dimsnowfallID']),
        ForeignKeyConstraint(['wind_id'], ['DimWind.dimwindID']),
        ForeignKeyConstraint(['humidity_id'], ['DimHumidity.dimhumidityID']),
    )

# @profile
def main():
    # Charger les données transformées
    df = pd.read_csv("/home/FIL_ROUGE/Historical/data/processed/weatherAirSoil.csv")

    # Convertir la colonne "time" en type datetime 
    df['time'] = pd.to_datetime(df['time'], format='%Y-%m-%d', errors='coerce')
    # df['time'] = pd.to_datetime(df['time'], unit='s')  # Convertir en datetime
    df['time'] = df['time'].dt.strftime('%Y-%m-%d')  # Formater en string
    df['time'] = pd.to_datetime(df['time'], format='%Y-%m-%d', errors='coerce')


    # Créer la chaîne de connexion SQLAlchemy
    server = '172.16.11.132'
    database = 'weather_warehouse'
    username = 'fidele'
    password = 'admin'
    conn_str = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server'

    # Créer le moteur SQLAlchemy
    engine = create_engine(conn_str, echo=False)

    # Créer la session SQLAlchemy
    session = Session(engine)

    try:
        conn = engine.connect()
        print("Connexion réussie.")

        # Créer les tables de dimension
        # Colonnes et tables de dimension
        dimension_columns = ['time', 'cloud_cover (%)', 'et0_fao_evapotranspiration (mm)',
                      'temperature_2m (°C)', 'vapour_pressure_deficit (kPa)', 'rain (mm)',
                      'snowfall (cm)', 'wind_speed_10m (m/s)', 'relative_humidity_2m (%)']

        dimension_tables = ['DimDate', 'DimCloudCover', 'DimEvapotranspiration',
                     'DimTemperature', 'DimVPD', 'DimRain', 'DimSnowfall', 'DimWind', 'DimHumidity']

        # Créer les tables de dimension
        dimension_dataframes = {}

        for column, table in zip(dimension_columns, dimension_tables):
            df_dimension = df[[column]].drop_duplicates().reset_index(drop=True)
            # Correction du nom de la colonne générée
            column_name = f"{table.lower()}ID"
            df_dimension.insert(0, column_name, range(1, len(df_dimension) + 1))
            df_dimension.to_sql(table, conn, if_exists='append', index=False, method='multi', chunksize=500)

            # Ajouter le DataFrame à dimension_dataframes
            dimension_dataframes[table] = df_dimension
            
        print("Tables de dimension !")

        # Créer le DataFrame df_fact
        df_fact = df.copy()

    
        # Utiliser dimension_dataframes pour ajouter les clés étrangères au DataFrame df_fact
        for table_name in dimension_tables:
            dimension_df = dimension_dataframes[table_name]
            common_cols = set(df_fact.columns) & set(dimension_df.columns)
            df_fact = pd.merge(df_fact, dimension_df, how='left', left_on=list(common_cols), right_on=list(common_cols))
            df_fact = df_fact.drop(columns=[col for col in dimension_df.columns if col != f"{table_name.lower()}ID"])
  

        # Renommer les colonnes dans le DataFrame df_fact
        df_fact = df_fact.rename(columns={'dimdateID': 'time_id', 'dimcloudcoverID': 'cloud_cover_id',
                                 'dimevapotranspirationID': 'evapotranspiration_id',
                                 'dimtemperatureID': 'temperature_id', 'dimvpdID': 'vpd_id',
                                 'dimrainID': 'rain_id', 'dimsnowfallID': 'snowfall_id',
                                 'dimwindID': 'wind_id', 'dimhumidityID': 'humidity_id'})

        # Supprimer les colonnes redondantes après l'ajout des clés étrangères
        columns_to_drop = [f"{table.lower()}ID" for table in dimension_tables if f"{table.lower()}ID" in df_fact.columns]
        df_fact = df_fact.drop(columns=columns_to_drop)

        # Insérer les données dans la table de fait avec les clés étrangères
        df_fact.to_sql('Fact_Air_Soil', conn, if_exists='append', index=False, method='multi', chunksize=100)

        # Commit des changements
        session.commit()
        print("Inserted Successfully !")

    except Exception as e:
        print(f"Erreur: {e}")

    finally:
        if session:
            session.close()

if __name__ == "__main__":
    main()

Connexion réussie.
Tables de dimension !
Inserted Successfully !


## Air Data Mart

In [2]:
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, Float, Date, ForeignKey, ForeignKeyConstraint
from sqlalchemy.orm import Session, declarative_base, relationship
import sys

Base = declarative_base()

class DimDate(Base):
    __tablename__ = 'DimDate'
    dimdateID = Column(Integer, primary_key=True, autoincrement=True)
    time = Column(Date, unique=True)

class DimHumidity(Base):
    __tablename__ = 'DimHumidity'
    dimhumidityID = Column(Integer, primary_key=True, autoincrement=True)
    relative_humidity_2m = Column(Float)

class DimCloudCover(Base):
    __tablename__ = 'DimCloudCover'
    dimcloudcoverID = Column(Integer, primary_key=True, autoincrement=True)
    cloud_cover = Column(Float)

class DimEvapotranspiration(Base):
    __tablename__ = 'DimEvapotranspiration'
    dimevapotranspirationID = Column(Integer, primary_key=True, autoincrement=True)
    et0_fao_evapotranspiration = Column(Float)

class DimTemperature(Base):
    __tablename__ = 'DimTemperature'
    dimtemperatureID = Column(Integer, primary_key=True, autoincrement=True)
    temperature_2m = Column(Float)

class DimVPD(Base):
    __tablename__ = 'DimVPD'
    dimvpdID = Column(Integer, primary_key=True, autoincrement=True)
    vapour_pressure_deficit = Column(Float)

class DimRain(Base):
    __tablename__ = 'DimRain'
    dimrainID = Column(Integer, primary_key=True, autoincrement=True)
    rain = Column(Float)

class DimSnowfall(Base):
    __tablename__ = 'DimSnowfall'
    dimsnowfallID = Column(Integer, primary_key=True, autoincrement=True)
    snowfall = Column(Float)
    snow_depth = Column(Float)

class DimWind(Base):
    __tablename__ = 'DimWind'
    dimwindID = Column(Integer, primary_key=True, autoincrement=True)
    wind_speed_10m = Column(Float)


class FactAir(Base):
    __tablename__ = 'Fact_Air'
    fact_ID = Column(Integer, primary_key=True, autoincrement=True)
    time_id = Column(Integer, ForeignKey('DimDate.dimdateID'))
    cloud_cover_id = Column(Integer, ForeignKey('DimCloudCover.dimcloudcoverID'))
    evapotranspiration_id = Column(Integer, ForeignKey('DimEvapotranspiration.dimevapotranspirationID'))
    temperature_id = Column(Integer, ForeignKey('DimTemperature.dimtemperatureID'))
    vpd_id = Column(Integer, ForeignKey('DimVPD.dimvpdID'))
    rain_id = Column(Integer, ForeignKey('DimRain.dimrainID'))
    snowfall_id = Column(Integer, ForeignKey('DimSnowfall.dimsnowfallID'))
    wind_id = Column(Integer, ForeignKey('DimWind.dimwindID'))
    humidity_id = Column(Integer, ForeignKey('DimHumidity.dimhumidityID'))

    # Ajoutez des mesures
    pm10 = Column(Float)
    pm2_5 = Column(Float)
    carbon_monoxide = Column(Float)
    nitrogen_dioxide = Column(Float)
    sulphur_dioxide = Column(Float)
    ozone = Column(Float)

    # Ajoutez les relations avec les tables de dimension
    time = relationship('DimDate', foreign_keys=[time_id])
    cloud_cover = relationship('DimCloudCover', foreign_keys=[cloud_cover_id])
    evapotranspiration = relationship('DimEvapotranspiration', foreign_keys=[evapotranspiration_id])
    temperature = relationship('DimTemperature', foreign_keys=[temperature_id])
    vpd = relationship('DimVPD', foreign_keys=[vpd_id])
    rain = relationship('DimRain', foreign_keys=[rain_id])
    snowfall = relationship('DimSnowfall', foreign_keys=[snowfall_id])
    wind = relationship('DimWind', foreign_keys=[wind_id])
    humidity = relationship('DimHumidity', foreign_keys=[humidity_id])

    # Définissez les contraintes de clé étrangère ici
    __table_args__ = (
        ForeignKeyConstraint(['cloud_cover_id'], ['DimCloudCover.dimcloudcoverID']),
        ForeignKeyConstraint(['time_id'], ['DimDate.dimdateID']),
        ForeignKeyConstraint(['evapotranspiration_id'], ['DimEvapotranspiration.dimevapotranspirationID']),
        ForeignKeyConstraint(['temperature_id'], ['DimTemperature.dimtemperatureID']),
        ForeignKeyConstraint(['vpd_id'], ['DimVPD.dimvpdID']),
        ForeignKeyConstraint(['rain_id'], ['DimRain.dimrainID']),
        ForeignKeyConstraint(['snowfall_id'], ['DimSnowfall.dimsnowfallID']),
        ForeignKeyConstraint(['wind_id'], ['DimWind.dimwindID']),
        ForeignKeyConstraint(['humidity_id'], ['DimHumidity.dimhumidityID']),
    )

# @profile
def main():
    # Charger les données transformées
    df = pd.read_csv("/home/FIL_ROUGE/Historical/data/processed/weatherAirSoil.csv")

    # Convertir la colonne "time" en type datetime 
    df['time'] = pd.to_datetime(df['time'], format='%Y-%m-%d', errors='coerce') 

    # Créer la chaîne de connexion SQLAlchemy
    server = '172.16.11.132'
    database = 'air_datamart'
    username = 'fidele'
    password = 'admin'
    conn_str = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server'

    # Créer le moteur SQLAlchemy
    engine = create_engine(conn_str, echo=False)

    # Créer la session SQLAlchemy
    session = Session(engine)

    try:
        conn = engine.connect()
        print("Connexion réussie.")

        # Créer les tables de dimension
        # Colonnes et tables de dimension
        dimension_columns = ['time', 'cloud_cover (%)', 'et0_fao_evapotranspiration (mm)',
                      'temperature_2m (°C)', 'vapour_pressure_deficit (kPa)', 'rain (mm)',
                      'snowfall (cm)', 'wind_speed_10m (m/s)', 'relative_humidity_2m (%)']

        dimension_tables = ['DimDate', 'DimCloudCover', 'DimEvapotranspiration',
                     'DimTemperature', 'DimVPD', 'DimRain', 'DimSnowfall', 'DimWind', 'DimHumidity']

        # Créer les tables de dimension
        dimension_dataframes = {}

        for column, table in zip(dimension_columns, dimension_tables):
            df_dimension = df[[column]].drop_duplicates().reset_index(drop=True)
            # Correction du nom de la colonne générée
            column_name = f"{table.lower()}ID"
            df_dimension.insert(0, column_name, range(1, len(df_dimension) + 1))
            df_dimension.to_sql(table, conn, if_exists='append', index=False, method='multi', chunksize=500)

            # Ajouter le DataFrame à dimension_dataframes
            dimension_dataframes[table] = df_dimension
            
        print("Tables de dimension AirDatamart !")

        # Créer le DataFrame df_fact
        df_fact = df.copy()

    
        # Utiliser dimension_dataframes pour ajouter les clés étrangères au DataFrame df_fact
        for table_name in dimension_tables:
            dimension_df = dimension_dataframes[table_name]
            common_cols = set(df_fact.columns) & set(dimension_df.columns)
            df_fact = pd.merge(df_fact, dimension_df, how='left', left_on=list(common_cols), right_on=list(common_cols))
            df_fact = df_fact.drop(columns=[col for col in dimension_df.columns if col != f"{table_name.lower()}ID"])
  

        # Renommer les colonnes dans le DataFrame df_fact
        df_fact = df_fact.rename(columns={'dimdateID': 'time_id', 'dimcloudcoverID': 'cloud_cover_id',
                                 'dimevapotranspirationID': 'evapotranspiration_id',
                                 'dimtemperatureID': 'temperature_id', 'dimvpdID': 'vpd_id',
                                 'dimrainID': 'rain_id', 'dimsnowfallID': 'snowfall_id',
                                 'dimwindID': 'wind_id', 'dimhumidityID': 'humidity_id'})

        # Supprimer les colonnes redondantes après l'ajout des clés étrangères
        columns_to_drop = [f"{table.lower()}ID" for table in dimension_tables if f"{table.lower()}ID" in df_fact.columns]
        df_fact = df_fact.drop(columns=columns_to_drop)

        # Insérer les données dans la table de fait avec les clés étrangères
        df_fact.to_sql('Fact_Air', conn, if_exists='append', index=False, method='multi', chunksize=100)

        # Commit des changements
        session.commit()
        print("Inserted Successfully AirDatamart!")

    except Exception as e:
        print(f"Erreur: {e}")

    finally:
        if session:
            session.close()

if __name__ == "__main__":
    main()

Connexion réussie.
Tables de dimension AirDatamart !
Inserted Successfully AirDatamart!


## Soil Data Mart

In [3]:
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, Float, Date, ForeignKey, ForeignKeyConstraint
from sqlalchemy.orm import Session, declarative_base, relationship
import sys

Base = declarative_base()

class DimDate(Base):
    __tablename__ = 'DimDate'
    dimdateID = Column(Integer, primary_key=True, autoincrement=True)
    time = Column(Date, unique=True)

class DimHumidity(Base):
    __tablename__ = 'DimHumidity'
    dimhumidityID = Column(Integer, primary_key=True, autoincrement=True)
    relative_humidity_2m = Column(Float)

class DimCloudCover(Base):
    __tablename__ = 'DimCloudCover'
    dimcloudcoverID = Column(Integer, primary_key=True, autoincrement=True)
    cloud_cover = Column(Float)

class DimEvapotranspiration(Base):
    __tablename__ = 'DimEvapotranspiration'
    dimevapotranspirationID = Column(Integer, primary_key=True, autoincrement=True)
    et0_fao_evapotranspiration = Column(Float)

class DimTemperature(Base):
    __tablename__ = 'DimTemperature'
    dimtemperatureID = Column(Integer, primary_key=True, autoincrement=True)
    temperature_2m = Column(Float)

class DimVPD(Base):
    __tablename__ = 'DimVPD'
    dimvpdID = Column(Integer, primary_key=True, autoincrement=True)
    vapour_pressure_deficit = Column(Float)

class DimRain(Base):
    __tablename__ = 'DimRain'
    dimrainID = Column(Integer, primary_key=True, autoincrement=True)
    rain = Column(Float)

class DimSnowfall(Base):
    __tablename__ = 'DimSnowfall'
    dimsnowfallID = Column(Integer, primary_key=True, autoincrement=True)
    snowfall = Column(Float)
    snow_depth = Column(Float)

class DimWind(Base):
    __tablename__ = 'DimWind'
    dimwindID = Column(Integer, primary_key=True, autoincrement=True)
    wind_speed_10m = Column(Float)


class FactSoil(Base):
    __tablename__ = 'Fact_Soil'
    fact_ID = Column(Integer, primary_key=True, autoincrement=True)
    time_id = Column(Integer, ForeignKey('DimDate.dimdateID'))
    cloud_cover_id = Column(Integer, ForeignKey('DimCloudCover.dimcloudcoverID'))
    evapotranspiration_id = Column(Integer, ForeignKey('DimEvapotranspiration.dimevapotranspirationID'))
    temperature_id = Column(Integer, ForeignKey('DimTemperature.dimtemperatureID'))
    vpd_id = Column(Integer, ForeignKey('DimVPD.dimvpdID'))
    rain_id = Column(Integer, ForeignKey('DimRain.dimrainID'))
    snowfall_id = Column(Integer, ForeignKey('DimSnowfall.dimsnowfallID'))
    wind_id = Column(Integer, ForeignKey('DimWind.dimwindID'))
    humidity_id = Column(Integer, ForeignKey('DimHumidity.dimhumidityID'))

    # Ajoutez des mesures
    soil_temperature_0_to_7cm = Column(Float)
    soil_temperature_7_to_28cm = Column(Float)
    soil_temperature_28_to_100cm = Column(Float)
    soil_temperature_100_to_255cm = Column(Float)
    soil_moisture_0_to_7cm = Column(Float)
    soil_moisture_7_to_28cm = Column(Float)
    soil_moisture_28_to_100cm = Column(Float)
    soil_moisture_100_to_255cm = Column(Float)

    # Ajoutez les relations avec les tables de dimension
    time = relationship('DimDate', foreign_keys=[time_id])
    cloud_cover = relationship('DimCloudCover', foreign_keys=[cloud_cover_id])
    evapotranspiration = relationship('DimEvapotranspiration', foreign_keys=[evapotranspiration_id])
    temperature = relationship('DimTemperature', foreign_keys=[temperature_id])
    vpd = relationship('DimVPD', foreign_keys=[vpd_id])
    rain = relationship('DimRain', foreign_keys=[rain_id])
    snowfall = relationship('DimSnowfall', foreign_keys=[snowfall_id])
    wind = relationship('DimWind', foreign_keys=[wind_id])
    humidity = relationship('DimHumidity', foreign_keys=[humidity_id])

    # Définissez les contraintes de clé étrangère ici
    __table_args__ = (
        ForeignKeyConstraint(['cloud_cover_id'], ['DimCloudCover.dimcloudcoverID']),
        ForeignKeyConstraint(['time_id'], ['DimDate.dimdateID']),
        ForeignKeyConstraint(['evapotranspiration_id'], ['DimEvapotranspiration.dimevapotranspirationID']),
        ForeignKeyConstraint(['temperature_id'], ['DimTemperature.dimtemperatureID']),
        ForeignKeyConstraint(['vpd_id'], ['DimVPD.dimvpdID']),
        ForeignKeyConstraint(['rain_id'], ['DimRain.dimrainID']),
        ForeignKeyConstraint(['snowfall_id'], ['DimSnowfall.dimsnowfallID']),
        ForeignKeyConstraint(['wind_id'], ['DimWind.dimwindID']),
        ForeignKeyConstraint(['humidity_id'], ['DimHumidity.dimhumidityID']),
    )

# @profile
def main():
    # Charger les données transformées
    df = pd.read_csv("/home/FIL_ROUGE/Historical/data/processed/weatherAirSoil.csv")

    # Convertir la colonne "time" en type datetime 
    df['time'] = pd.to_datetime(df['time'], format='%Y-%m-%d', errors='coerce')

    # Créer la chaîne de connexion SQLAlchemy
    server = '172.16.11.132'
    database = 'soil_datamart'
    username = 'fidele'
    password = 'admin'
    conn_str = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server'

    # Créer le moteur SQLAlchemy
    engine = create_engine(conn_str, echo=False)

    # Créer la session SQLAlchemy
    session = Session(engine)

    try:
        conn = engine.connect()
        print("Connexion réussie.")

        # Créer les tables de dimension
        # Colonnes et tables de dimension
        dimension_columns = ['time', 'cloud_cover (%)', 'et0_fao_evapotranspiration (mm)',
                      'temperature_2m (°C)', 'vapour_pressure_deficit (kPa)', 'rain (mm)',
                      'snowfall (cm)', 'wind_speed_10m (m/s)', 'relative_humidity_2m (%)']

        dimension_tables = ['DimDate', 'DimCloudCover', 'DimEvapotranspiration',
                     'DimTemperature', 'DimVPD', 'DimRain', 'DimSnowfall', 'DimWind', 'DimHumidity']

        # Créer les tables de dimension
        dimension_dataframes = {}

        for column, table in zip(dimension_columns, dimension_tables):
            df_dimension = df[[column]].drop_duplicates().reset_index(drop=True)
            # Correction du nom de la colonne générée
            column_name = f"{table.lower()}ID"
            df_dimension.insert(0, column_name, range(1, len(df_dimension) + 1))
            df_dimension.to_sql(table, conn, if_exists='append', index=False, method='multi', chunksize=500)

            # Ajouter le DataFrame à dimension_dataframes
            dimension_dataframes[table] = df_dimension
            
        print("Tables de dimension SoilDatamrt!")

        # Créer le DataFrame df_fact
        df_fact = df.copy()

    
        # Utiliser dimension_dataframes pour ajouter les clés étrangères au DataFrame df_fact
        for table_name in dimension_tables:
            dimension_df = dimension_dataframes[table_name]
            common_cols = set(df_fact.columns) & set(dimension_df.columns)
            df_fact = pd.merge(df_fact, dimension_df, how='left', left_on=list(common_cols), right_on=list(common_cols))
            df_fact = df_fact.drop(columns=[col for col in dimension_df.columns if col != f"{table_name.lower()}ID"])
  

        # Renommer les colonnes dans le DataFrame df_fact
        df_fact = df_fact.rename(columns={'dimdateID': 'time_id', 'dimcloudcoverID': 'cloud_cover_id',
                                 'dimevapotranspirationID': 'evapotranspiration_id',
                                 'dimtemperatureID': 'temperature_id', 'dimvpdID': 'vpd_id',
                                 'dimrainID': 'rain_id', 'dimsnowfallID': 'snowfall_id',
                                 'dimwindID': 'wind_id', 'dimhumidityID': 'humidity_id'})

        # Supprimer les colonnes redondantes après l'ajout des clés étrangères
        columns_to_drop = [f"{table.lower()}ID" for table in dimension_tables if f"{table.lower()}ID" in df_fact.columns]
        df_fact = df_fact.drop(columns=columns_to_drop)

        # Insérer les données dans la table de fait avec les clés étrangères
        df_fact.to_sql('Fact_Soil', conn, if_exists='append', index=False, method='multi', chunksize=100)

        # Commit des changements
        session.commit()
        print("Inserted Successfully SoilDatamart!")

    except Exception as e:
        print(f"Erreur: {e}")

    finally:
        if session:
            session.close()

if __name__ == "__main__":
    main()

Connexion réussie.
Tables de dimension SoilDatamrt!
Inserted Successfully SoilDatamart!
